In [163]:
import pandas as pd
import requests, re, time
from bs4 import BeautifulSoup

In [58]:
url = "https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A2811119011%2Cn%3A7072561011&dc&ds=v1%3ALg%2BI7W8x3qOCmrDsTOUseTdkjVGEHCRYQPlZKCjiQlo&qid=1662655090&rnid=2811119011&ref=sr_nr_n_1"

In [59]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62',
            'Accept-Language': 'en-US, en;q=0.5'})

In [60]:
# getting the url response

response = requests.get(url=url, headers=headers)

In [61]:
# getting the response content

primaryPageSoup = BeautifulSoup(markup=response.content, features='lxml')

In [62]:
# finding the container 

parentDiv = primaryPageSoup.find(name="div", 
                                attrs={'class': 's-main-slot s-result-list s-search-results sg-row'})

In [63]:
# finding the card with phone titles

phoneDivs = parentDiv.findChildren(name="div", attrs={'data-uuid': True},
                                   recursive=False)

Finding children <b>recursively</b> entails getting *their* grandchildren too, as _viable_ results.

In [64]:
# Getting title information

titleCard = phoneDivs[0].find(name='h2').text
print(titleCard)

Nokia XR20 5G | Android 11 | Unlocked Smartphone | Dual SIM | US Version | 6/128GB | 6.67-Inch Screen | 48MP Dual Camera | Polar Night  


In [65]:
redirectUrl = phoneDivs[0].find(name='a')['href']

In [66]:
mainUrl = 'https://www.amazon.com'

In [67]:
pageRedirect = requests.get(mainUrl + redirectUrl, headers=headers)

In [68]:
redirectSoup = BeautifulSoup(markup=pageRedirect.content, features='lxml')

In [69]:
infoTable = redirectSoup.find(name='table', 
                              attrs={'class': 'a-normal a-spacing-micro'})

In [70]:
brandRow = infoTable.find(name='tr', 
                          attrs={'class': 'a-spacing-small po-brand'})

In [71]:
brand = brandRow.findChildren(name='td', recursive=False)[1].text.strip()

In [72]:
modelRow = infoTable.find(name='tr', 
                          attrs={'class': 'a-spacing-small po-model_name'})

In [73]:
model = modelRow.findChildren(name='td', recursive=False)[1].text.strip()

In [74]:
opSystemrow = infoTable.find(name='tr', 
                             attrs={'class': 'a-spacing-small po-operating_system'})

In [75]:
operatingSystem = opSystemrow.findChildren(name='td', recursive=False)[1].text.strip()

In [76]:
memoryStorageRow = infoTable.find(name='tr', 
                                  attrs={'class': 'a-spacing-small po-memory_storage_capacity'})

In [77]:
memoryStorage = memoryStorageRow.findChildren(name='td', recursive=False)[1].text.strip()

In [78]:
displaySizeRow = infoTable.find(name='tr',
                                attrs={'class': 'a-spacing-small po-display.size'})

In [79]:
displaySize = displaySizeRow.findChildren(name="td", recursive=False)[1].text.strip()

In [80]:
connectorTypeRow = infoTable.find(name="tr", attrs={'class': 'a-spacing-small po-connector_type'})

In [81]:
connectorType = connectorTypeRow.findChildren(name="td", recursive=False)[1].text.strip()

AttributeError: 'NoneType' object has no attribute 'findChildren'

In [82]:
print(model, brand, memoryStorage)

Nokia XR20 Nokia 128 GB


We can generalize this process for each card and collect:
- the brand name
- the model of the device
- the memory storage
- the screen/display size, and
- the connector type

Ratings by <b>*features*</b>.
> to be implemented

In [103]:
# Grabbing link to reviews
reviewsUrl = redirectSoup.find(name="a", 
                               attrs={'data-hook': 'see-all-reviews-link-foot'})['href']

## Generalizaing Pagination for Reviews

In [175]:
# Pagination
pageCounter = 0
collectedReviews = []

while pageCounter < 5:
    nextUrl = reviewsUrl
    # Getting request for the review page
    reviewsPage = requests.get(url=mainUrl + reviewsUrl,
                          headers=headers)
    # Creating the Soup
    reviewsSoup = BeautifulSoup(reviewsPage.content, features='lxml')
    
    # Obtaining the reviews parent div
    reviewsDiv = reviewsSoup.find(name="div", 
                               attrs={'id': 'cm_cr-review_list'})
    
    # All reviews in their respective divs
    reviewsList = reviewsDiv.findChildren(name='div',
                                     attrs={'data-hook': 'review'},
                                     recursive=False)
    for reviewDivs in reviewsList:
        text = reviewsDiv.find(name="span", attrs={'data-hook': 'review-body'}).text.strip('\n')
        collectedReviews.append(text)
    try:   
        # Getting next page url
        paginationContainer = reviewsSoup.find(name="ul", attrs={'class': 'a-pagination'})
        nextUrl = paginationContainer.findChildren(name="li", recursive=False)[1].find(name="a")['href']
    except AttributeError:
        break
    
    # Sleeping to avoid making too many requests
    time.sleep(2)
    
    # Updating the page counter
    pageCounter += 1

In [165]:
collectedReviews.__len__()

50